## ENGI E1006: Introduction to Computing for Engineers and Applied Scientists
---

In this section, we'll go into a more detailed implementation of the K Nearest Neighbors algorithm using `numpy`. 

Recall from lecture that the goal of this algorithm is to classify an unknown data point by finding its `K` closest known data points, and taking a vote. We will implement this algorithm on the `Iris` dataset from last time.

Mathematically, this algorithm is fairly straightforward. The only known calculation we need is Euclidean Distance in N dimensional space. in $R^2$, we can visualize the formula with a triangle:

<img src="assets/euclidean_distance.png" width=200></img>

For the Iris dataset, we will be in $R^4$, and so have to use the more general formula for $R^n$:
$$ \sqrt{\sum_{i=1}^{n} (x_i - y_i)^2} $$

Note that depending on the nature of our dataset, we could've chosen a different distance, such as Manhattan Distance:
$$ \sum_{i=1}^{n} |x_i - y_i| $$



In [ ]:
import numpy as np
import sklearn # notice that scikit-learn's import name is sklearn!
from sklearn.datasets import load_iris

In [ ]:
iris_data = load_iris()

In [ ]:
# join the data vertically so that the targets are a new column
all_data = np.concatenate((iris_data.data, iris_data.target.reshape((len(iris_data.target), 1))), axis=1)

# shuffle
np.random.shuffle(all_data)
all_data

In [ ]:
def split_dataset(X, test_percentage=.2):
    # Split into train and test
    trX = X[int(test_percentage*len(X)):]
    tsX = X[:int(test_percentage*len(X))]

    # separate the labels
    trY = trX[:,-1:] # grab just the labels column
    trY = trY.reshape(len(trY)) # reshape as vector
    trX = trX[:,:-1] # slice out the labels column

    tsY = tsX[:,-1:] # grab just the labels column
    tsY = tsY.reshape(len(tsY)) # reshape as vector
    tsX = tsX[:,:-1] # slice out the labels column
    
    return trX, trY, tsX, tsY

In [ ]:
train_data, train_data_labels, test_data, test_data_labels = \
    split_dataset(all_data, .2)

In [ ]:
train_data.shape

In [ ]:
train_data_labels.shape

In [ ]:
test_data.shape

In [ ]:
test_data_labels.shape

In [ ]:
# Now lets start implementing our function
from statistics import mode
from math import sqrt


def knn(data, data_labels, vector, k):
    # data is the "known" points
    # data_labels are their labels
    # vector is an "unknown" point
    
    # preallocate distance array
    distances = np.zeros(len(data_labels))

    # calculate distances
    for i in range(len(distances)):
        distances[i] = sqrt(((data[i].astype(float) - vector.astype(float)) ** 2).sum())

    # set labels
    indexes = np.argsort(distances)

    # take vote amongs top labels
    to_vote = data_labels[indexes]
    return mode(to_vote[:k])



In [ ]:
# lets make sure this works!

correct = 0

for _ in range(len(test_data_labels)):
    predicted_label = knn(train_data, train_data_labels, test_data[_], 5)
    real_label = test_data_labels[_]
    
    print('Predicted Label: {}\t Real Label: {}'.format(
        predicted_label,
        real_label)
     )
    
    if real_label == predicted_label:
        correct += 1

print("Accuracy: {}%".format(
        int(correct/len(test_data_labels) * 100)
    )
)